In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
%matplotlib inline

In [20]:
df = pd.read_csv('alpha24_step15Stud.csv')
df.head()

,czas(min),alpha,temp,Ts
0,0,1,"6,50",2
1,15,0,"6,35","2,18"
2,30,0,"6,20","2,19"
3,45,1,"6,05","1,68"
4,60,1,"5,90","1,5"


In [21]:
df['temp'] = df['temp'].apply(lambda x: x.replace(',','.')).astype('float64')
df['Ts'] = df['Ts'].apply(lambda x: x.replace(',','.')).astype('float64')

In [22]:
df.head()

,czas(min),alpha,temp,Ts
0,0,1,6.50,2.00
1,15,0,6.35,2.18
2,30,0,6.20,2.19
3,45,1,6.05,1.68
4,60,1,5.90,1.50


In [23]:
df.rename(columns={"czas(min)": "time", "alpha": "compressor", "temp": 'ambient_temp', "Ts": 'refrigerator_temp'}, inplace=True)
df.set_index('time', inplace=True)
df.head()

,compressor,ambient_temp,refrigerator_temp
time,,,
0,1,6.50,2.00
15,0,6.35,2.18
30,0,6.20,2.19
45,1,6.05,1.68
60,1,5.90,1.50


In [24]:
size = 1000
delay = 100

a = df.iloc[0:size,:].to_numpy().reshape(delay,size*3//delay)
a

array([[ 1.  ,  6.5 ,  2.  , ...,  0.  ,  5.26,  1.82],
       [ 0.  ,  5.23,  1.86, ...,  0.  ,  5.39,  1.48],
       [ 0.  ,  5.3 ,  1.53, ...,  1.  ,  5.99,  0.9 ],
       ...,
       [ 0.  ,  1.97, -1.98, ...,  0.  ,  1.23, -1.93],
       [ 0.  ,  1.2 , -1.92, ...,  0.  ,  2.01, -2.07],
       [ 1.  ,  2.13, -2.17, ...,  1.  ,  4.01, -2.48]])

In [25]:
from random_forest.tree import Tree
import random_forest

random_forest.tree.config.min_split_size = 30
random_forest.tree.config.number_of_predictors_to_draw = 10

In [26]:
myTree = Tree(a)
test_data = df.iloc[1000:1010,:].to_numpy().reshape(30)
print(test_data, myTree.predict(test_data))

[ 1.    4.2  -2.5   1.    4.39 -2.51  1.    4.57 -2.52  1.    4.74 -2.53
  1.    4.9  -2.54  1.    5.05 -2.55  1.    5.18 -2.56  1.    5.3  -2.57
  1.    5.4  -2.57  0.    5.35 -2.35] -2.6805263157894736


In [27]:
print(myTree)


        min_split_size: 50
        number_of_predictors_to_draw: 10
        
        Branch:
            if <= 26_-1.15:
                
        Branch:
            if <= 20_-2.33:
                Leaf: -2.6805263157894736
            else:
                Leaf: -1.76125  
        
            else:
                Leaf: 0.004489795918367351  
        
        
